# ACO for Optimal Power Flow

ACO digunakan untuk merancang distribusi daya yang efisien dengan memecahkan masalah aliran daya optimal dalam jaringan listrik sehingga meminimalisir biaya sekaligus memenuhi kebutuhan listrik.


## Inisiasi Parameter

In [ ]:
import numpy as np
import random

# Parameter ACO
NUM_ANTS = 10
NUM_ITERATIONS = 50
ALPHA = 1       # Pengaruh feromon
BETA = 2        # Pengaruh heuristik
EVAPORATION = 0.5
Q = 1           # Konstanta feromon

*   NUM_ANTS = 10

Jumlah semut yang digunakan dalam setiap iterasi dimana semut berperan sebagai agen pencari solusi. Semakin banyak semut maka eksplorasi solusi lebih luas namun waktu komputasi meningkat. Nilai 10 dipilih karena data merupakan dummy sederhana sebagai simulasi.

*   NUM_ITERATIONS = 50

Jumlah iterasi yang dijalankan algoritma. Semakin besar jumlah iterasi makan kemungkinan menemukan solusi terbaik meningkat tetapi memakan biaya waktu komputasi.

*   ALPHA = 1

Pengaruh feromon dalam probabilitas pemilihan jalur.Semakin besar 𝛼 maka jalur dengan feromon lebih banyak diprioritaskan (eksploitasi).

*   BETA = 2

Pengaruh heuristik (informasi lokal) dalam probabilitas pemilihan jalur. Semakin besar 𝛽 maka lebih mengutamakan jalur dengan nilai heuristik tinggi (eksplorasi).

*   EVAPORATION = 0.5

Laju penguapan feromon di setiap iterasi (𝜌) . Nilai kecil (𝜌 → 0) maka feromon bertahan lama (eksploitasi). Nilai besar (𝜌 → 1) maka feromon cepat menghilang (eksplorasi).

*   Q = 1

Konstanta feromon yang menentukan seberapa banyak feromon ditambahkan ke jalur oleh agen. Nilai Q besar memperkuat jalur yang baik secara signifikan. Sedangkan nilai Q kecil membuat eksplorasi lebih merata.

## Input Data Generator

Setiap pembangkit memiliki batas daya minimum dan maksimum (𝑃min,𝑃max) dan koefisien fungsi biaya.

In [ ]:
# Data sistem pembangkit
GEN_DATA = [
    {"Pmin": 10, "Pmax": 50, "cost_coeff": [0.2, 10, 5]},  # [a, b, c]
    {"Pmin": 20, "Pmax": 60, "cost_coeff": [0.15, 8, 7]},
    {"Pmin": 30, "Pmax": 80, "cost_coeff": [0.1, 6, 10]}]

TOTAL_DEMAND = 100  # Permintaan daya total (MW)
NUM_GENERATORS = len(GEN_DATA)

## Fungsi Biaya

In [ ]:
# Fungsi biaya pembangkitan daya
def generation_cost(power, coeff):
    a, b, c = coeff
    return a * power**2 + b * power + c

# Fungsi untuk menghitung total biaya
def total_cost(solution):
    cost = 0
    for i, power in enumerate(solution):
        cost += generation_cost(power, GEN_DATA[i]["cost_coeff"])
    return cost

Fungsi kuadratik C(P)=aP^2+bP+c digunakan untuk menghitung biaya pembangkit daya. Fungsi kuadratik dipakai karena bentuknya yang sederhana namun cukup fleksibel sehingga memungkinkan perhitungan efisien dalam algoritma optimasi serta dapat merepresentasikan dengan baik karakteristik biaya operasional sebagian besar pembangkit daya.

*   Komponen kuadrat (aP^2), Mewakili biaya yang meningkat secara non linear dengan daya yang dihasilkan. Biaya ini disebabkan oleh efisiensi turbin atau generator yang menurun ketika pembangkit beroperasi mendekati kapasitas maksimal.

*   Komponen Linear (bP), Menunjukkan biaya variabel seperti bahan bakar atau kebutuhan tenaga kerja tambahan yang meningkat sebanding dengan daya yang diharapkan.

*   Komponen konstanta (c), Merupakan biaya tetap yang tidak bergantung pada daya yang dihasilkan seperti biaya administrasi atau perawatan.

# Algoritma ACO
Setiap semut mencari solusi berdasarkan probabilitas pemilihan data untuk setiap pembangkit. Sedangkan feromon diperbarui berdasarkan solusi yang ditemukan oleh semut.

In [3]:
import numpy as np
import random

# Parameter ACO
NUM_ANTS = 10
NUM_ITERATIONS = 50
ALPHA = 1       # Pengaruh feromon
BETA = 2        # Pengaruh heuristik
EVAPORATION = 0.5
Q = 1           # Konstanta feromon

# Data sistem pembangkit
GEN_DATA = [
    {"Pmin": 10, "Pmax": 50, "cost_coeff": [0.2, 10, 5]},  # [a, b, c]
    {"Pmin": 20, "Pmax": 60, "cost_coeff": [0.15, 8, 7]},
    {"Pmin": 30, "Pmax": 80, "cost_coeff": [0.1, 6, 10]}
]

TOTAL_DEMAND = 100  # Permintaan daya total (MW)
NUM_GENERATORS = len(GEN_DATA)

# Fungsi biaya pembangkitan daya
def generation_cost(power, coeff):
    a, b, c = coeff
    return a * power**2 + b * power + c

# Fungsi untuk menghitung total biaya
def total_cost(solution):
    cost = 0
    for i, power in enumerate(solution):
        cost += generation_cost(power, GEN_DATA[i]["cost_coeff"])
    return cost

# Inisialisasi feromon
pheromone = np.ones((NUM_GENERATORS, 101))  # Untuk setiap pembangkit dan kemungkinan daya (0-100 MW)

# Algoritma ACO
def ant_colony_optimization():
    global pheromone
    best_solution = None
    best_cost = float('inf')

    for iteration in range(NUM_ITERATIONS):
        all_solutions = []
        all_costs = []

        # Setiap semut mencari solusi
        for ant in range(NUM_ANTS):
            solution = []
            remaining_demand = TOTAL_DEMAND

            # Setiap pembangkit menentukan daya
            for i in range(NUM_GENERATORS):
                # Probabilitas pemilihan daya
                probabilities = []
                for power in range(101):
                    if GEN_DATA[i]["Pmin"] <= power <= GEN_DATA[i]["Pmax"] and power <= remaining_demand:
                        heuristic = 1 / (generation_cost(power, GEN_DATA[i]["cost_coeff"]) + 1)
                        prob = (pheromone[i][power]**ALPHA) * (heuristic**BETA)
                    else:
                        prob = 0
                    probabilities.append(prob)
                probabilities = np.array(probabilities)
                probabilities = probabilities.sum()

                # Probabilitas pemilihan daya
                probabilities = []
                for power in range(101):
                    if GEN_DATA[i]["Pmin"] <= power <= GEN_DATA[i]["Pmax"] and power <= remaining_demand:
                        heuristic = 1 / (generation_cost(power, GEN_DATA[i]["cost_coeff"]) + 1)
                        prob = (pheromone[i][power]**ALPHA) * (heuristic**BETA)
                    else:
                        prob = 0
                    probabilities.append(prob)

                # Konversi ke array numpy
                probabilities = np.array(probabilities, dtype=float)

                # Tangani kasus jika semua probabilitas bernilai nol
                if probabilities.sum() == 0:
                    probabilities = np.ones_like(probabilities) / len(probabilities)  # Probabilitas seragam
                else:
                    probabilities /= probabilities.sum()  # Normalisasi

                # Pilih daya berdasarkan probabilitas
                #power = np.random.choice(range(101), p=probabilities)

                # Pilih daya berdasarkan probabilitas
                power = np.random.choice(range(101), p=probabilities)
                solution.append(power)
                remaining_demand -= power

            # Validasi solusi
            if remaining_demand <= 0:
                solution_cost = total_cost(solution)
                all_solutions.append(solution)
                all_costs.append(solution_cost)

                # Perbarui solusi terbaik
                if solution_cost < best_cost:
                    best_solution = solution
                    best_cost = solution_cost

        # Update feromon
        pheromone *= (1 - EVAPORATION)
        for i, solution in enumerate(all_solutions):
            for g, power in enumerate(solution):
                pheromone[g][power] += Q / all_costs[i]

        # Log iterasi
        print(f"Iterasi {iteration + 1}: Biaya terbaik = {best_cost}, Solusi terbaik = {best_solution}")

    return best_solution, best_cost

# Jalankan ACO
best_solution, best_cost = ant_colony_optimization()
print("\nSolusi optimal:")
for i, power in enumerate(best_solution):
    print(f"Pembangkit {i + 1}: {power} MW")
print(f"Total Biaya: {best_cost}")


Iterasi 1: Biaya terbaik = 1283.85, Solusi terbaik = [32, 21, 47]
Iterasi 2: Biaya terbaik = 1283.85, Solusi terbaik = [32, 21, 47]
Iterasi 3: Biaya terbaik = 1283.85, Solusi terbaik = [32, 21, 47]
Iterasi 4: Biaya terbaik = 1283.85, Solusi terbaik = [32, 21, 47]
Iterasi 5: Biaya terbaik = 1283.85, Solusi terbaik = [32, 21, 47]
Iterasi 6: Biaya terbaik = 1283.85, Solusi terbaik = [32, 21, 47]
Iterasi 7: Biaya terbaik = 1283.85, Solusi terbaik = [32, 21, 47]
Iterasi 8: Biaya terbaik = 1283.85, Solusi terbaik = [32, 21, 47]
Iterasi 9: Biaya terbaik = 1283.85, Solusi terbaik = [32, 21, 47]
Iterasi 10: Biaya terbaik = 1283.85, Solusi terbaik = [32, 21, 47]
Iterasi 11: Biaya terbaik = 1238.25, Solusi terbaik = [10, 29, 61]
Iterasi 12: Biaya terbaik = 1238.25, Solusi terbaik = [10, 29, 61]
Iterasi 13: Biaya terbaik = 1238.25, Solusi terbaik = [10, 29, 61]
Iterasi 14: Biaya terbaik = 1238.25, Solusi terbaik = [10, 29, 61]
Iterasi 15: Biaya terbaik = 1238.25, Solusi terbaik = [10, 29, 61]
Iter

## HASIL
Pembangkit 1 menghasilkan daya 10 MW

Pembangkit 2 menghasilkan daya 29 MW

Pembangkit 3 menghasilkan daya 61 MW

Kombinasi tersebut memenuhi permitaan daya sebesar 100 MW sekaligus menghasilkan biaya total terendah yaitu 1238.25 unit.

# Kesimpulan

Kode ini adalah implementasi dasar yang dapat dimodifikasi untuk menambah lebih banyak pembangkit, parameter biaya atau modifikasi menggunakan tools machine learning seperti GridSearch untuk hasil yang lebih baik. Implementasi ini juga dapat dikembangkan lebih lanjut dengan menggunakan train-test data untuk mengukur akurasinya.